## 참고한 사이트
* https://github.com/jsongcat/movie_recommendation_system

In [ ]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import combinations
import time
import random
import json
import pickle
import matplotlib.pyplot as plt

## 데이터 불러오기

In [2]:
data_path = os.path.abspath('../data')

In [3]:
df = pd.read_csv(data_path+'/ml-100k/u.data', delimiter='\t', names=['user','item','rating','time'])
num_users = len(np.unique(df['user']))
num_items = len(np.unique(df['item']))

In [4]:
def train_test_split(train_rate, ind_list, seed_num):
    """
    train과 test data로 나눔
    
    Parameters
    ----------
    train_rate : float
        train 데이터 비율
    ind_list : like list
        나누고자 하는 데이터의 index 값들
    seed_num : int
        랜덤 seed 번호
        
    Returns
    -------
    inds_train : list
        train에 해당하는 index 번호를 담은 list
    inds_test : list
        test에 해당하는 index 번호를 담은 list

    """
    random.seed(seed_num)
    
    n = len(ind_list)
    num_of_train = int(n*train_rate)
    inds_train = random.sample(ind_list,num_of_train)
    inds_test = list(set(ind_list).difference(inds_train))
    return inds_train, inds_test
    

In [ ]:
def make_default_dict(data_list):
    """
    user, item, rating 관계를 dict로 만듦
    
    Parameters
    ----------
    data_list : like list
        [user, item, rating]의 리스트 형태의 객체를 담은 list
        ex) [ [100, 1000, 3], [101, 1001, 4], ...]

    Returns
    -------
    user_dict : dict
        user 번호를 key값으로, user 번호의 key값에 해당하는 key값을 
        item으로 놓고 이에 해당하는 점수가 value 값이 됨
        ex) user_dict[100][1000] = 3 
          - 100 : user
          - 1000 : item
          - 3 : rating
        
    item_dict : dict
        item 번호를 key값으로, item 번호의 key값에 해당하는 key값을 
        user 놓고 이에 해당하는 점수가 value 값이 됨
        ex) user_dict[1000][100] = 3
          - 1000 : item
          - 100 : user
          - 3 : rating

    """
    user_dict = defaultdict(dict)
    item_dict = defaultdict(dict)
    
    for user,item,rating in data_list:
        user_dict[user][item] = rating
        item_dict[item][user] = rating
    
    return user_dict, item_dict

def make_mean_dict(dict_):
    """
    각 user나 item의 평균 점수값을 담은 dict 생성
    
    Parameters
    ----------
    dict_ : like dict
        user 번호를 key값으로, user 번호의 key값에 해당하는 key값을 
        item으로 놓고 이에 해당하는 점수가 value 값이 됨
        (반대로 item을 key값으로 하는 item_dict도 가능)
        ex) user_dict[100][1000] = 3

    Returns
    -------
    mean_dict : dict
        각 user 또는 item을 key값으로 하고 이에 대응하는 
        값은 해당 user 또는 item의 평균값임

    """
    # item 평균값 구하기
    mean_dict = {}
    for key in dict_.keys():
        target = dict_[key].values()
        if not target:
            continue
        mean_dict[key] = sum(target)/len(target)
    return mean_dict

## train과 test로 나누기 및 데이터 형태 변환
* data_list : list에 데이터를 [user, item, rating] 구조로 넣음  
  ex) [[100, 1000, 3], [101, 1001, 4], ...]
* tra_user_dict : user, item, rating을 dict 형태로 넣음. 여기선 user가 key값이 되며 tra는 train을 의미함  
  ex) tra_user_dict[100][1000] = 3 : user 100의 item 1000에 대한 점수는 3점
  * tra_item_dict : item을 key값으로 user, rating을 dict 형태로 넣음
* tra_user_mean_dict : 각 user를 key값, 각 user의 평균 점수 값을 value로 넣음

In [5]:
train_rate = 0.8
seed_num = 65535
inds_train, inds_test = train_test_split(train_rate, df.index.tolist(), seed_num)

In [6]:
train = df.loc[inds_train,:]
test = df.loc[inds_test,:]

In [8]:
data_list = df[['user','item','rating']].values.tolist()
train_data_list = train[['user','item','rating']].values.tolist()
test_data_list = test[['user','item','rating']].values.tolist()

In [9]:
tra_user_dict, tra_item_dict = make_default_dict(train_data_list)
tes_user_dict, tes_item_dict = make_default_dict(test_data_list)
tra_user_mean_dict = make_mean_dict(tra_user_dict)
tra_item_mean_dict = make_mean_dict(tra_item_dict)

In [10]:
def cal_correlate(item1_rating, item2_rating, item1_mean, item2_mean):
    """
    2개 item 간의 상관관계 계산
    
    Parameters
    ----------
    item1_rating : int
        item 1의 점수
    item2_rating : int
        item 2의 점수
    item1_mean : float or int
        item 1의 평균값
    item2_mean : float or int
        item 2의 평균값
    
    Returns
    -------
    numer : float
        식에서 분자에 해당하는 값
    denom1 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 1과 관련된 값)
    denom2 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 2과 관련된 값)
    """
    numer = (item1_rating-item1_mean)*(item2_rating-item2_mean)
    denom1 = (item1_rating-item1_mean)**2
    denom2 = (item2_rating-item2_mean)**2
    return numer, denom1, denom2

def cal_adjusted_cosine(item1_rating, item2_rating, user_mean):
    """
    2개 item 간의 조정된 cosine 값 계산
    
    Parameters
    ----------
    item1_rating : int
        item 1의 점수
    item2_rating : int
        item 2의 점수
    user_mean : float or int
        user의 점수 평균값
    
    Returns
    -------
    numer : float
        식에서 분자에 해당하는 값
    denom1 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 1과 관련된 값)
    denom2 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 2과 관련된 값)
    """
    numer = (item1_rating-user_mean)*(item2_rating-user_mean)
    denom1 = (item1_rating-user_mean)**2
    denom2 = (item2_rating-user_mean)**2
    return numer, denom1, denom2

def cal_cosine(item1_rating, item2_rating):
    """
    2개 item 간의 cosine 값 계산
    
    Parameters
    ----------
    item1_rating : int
        item 1의 점수
    item2_rating : int
        item 2의 점수
    
    Returns
    -------
    numer : float
        식에서 분자에 해당하는 값
    denom1 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 1과 관련된 값)
    denom2 : float
        식에서 분모의 왼쪽 부분에 해당하는 값(item 2과 관련된 값)
    """
    numer = item1_rating*item2_rating
    denom1 = item1_rating**2
    denom2 = item2_rating**2
    return numer, denom1, denom2

def cal_numer_denom(numer, denom1,denom2):
    """
    분모를 정규화 한뒤 분모와 분자 계산
    
    Parameters
    ----------
    numer : float
        특정 식에서 분자의 모든 값
    denom1 : float
        특정 식에서 분모의 모든 값(item1, 왼쪽 부분)
    denom2 : float
        특정 식에서 분모의 모든 값(item2, 오른쪽 부분)
    
    Returns
    -------
    result : float
        분모와 분자를 계산한 결과값
    """
    result = 0
    
    denom = (denom1**0.5)*(denom2**0.5)
    if denom!=0:
        result = numer/denom
    return result

def sort_dict(target_dict):
    """
    유사도를 내림차순으로 정렬함
    
    Parameters
    ----------
    target_dict : dict
        item 2개를 2중 key값으로 하여 두 item의 유사도 값을 value 값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    
    Returns
    -------
    target_dict : dict
        item 2개를 2중 key값으로 하여 두 item의 유사도 값을 value 값으로 가지는 dict로 유사도 값이 내림차순임
    """
    for item in target_dict.keys():
        a = sorted(target_dict[item].items(), key=(lambda x: x[1]), reverse=True)
        a_dict = {}
        for i in range(len(a)):
            a_dict[a[i][0]] = a[i][1]    
        target_dict[item] = a_dict
    return target_dict

In [11]:
def make_item_sim_dict(item_dict, user_mean_dict, item_mean_dict):
    """
    item 간 유사도를 계산함
    
    Parameters
    ----------
    item_dict : dict
        item 번호를 key값으로, item 번호의 key값에 해당하는 key값을 
        user 놓고 이에 해당하는 점수가 value 값이 됨
        ex) item_dict[1000][100] = 3
    user_mean_dict : dict
        user를 key값으로 하며 대응하는 value 값은 해당 user의 평균 rating(점수) 값임
    item_mean_dict : float
        item key값으로 하며 대응하는 value 값은 해당 user의 평균 rating(점수) 값임
    
    Returns
    -------
    cosine_dict : float
        item 2개를 2중 key값으로 하여 두 item의 cosine 유사도 값을 value 값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    adj_cosine_dict : dict
        item 2개를 2중 key값으로 하여 두 item의 adjusted cosine 유사도 값을 value 값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    corr_dict : dict
        item 2개를 2중 key값으로 하여 두 item의 correlation 유사도 값을 value 값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    """

    cosine_dict = defaultdict(dict)
    adj_cosine_dict = defaultdict(dict)
    corr_dict = defaultdict(dict)
    
    start = time.time()
    for item1 in item_dict.keys():
        for item2 in item_dict.keys():
            if item1 == item2 :
                continue
            
            elif item1 in cosine_dict[item2].keys() :
                cosine_dict[item1][item2] = cosine_dict[item2][item1]
                adj_cosine_dict[item1][item2] = adj_cosine_dict[item2][item1]
                corr_dict[item1][item2] = corr_dict[item2][item1]
                continue
            
            else:
                item1_users = item_dict[item1].keys()
                item2_users = item_dict[item2].keys()
                both_users = set(item1_users).intersection(item2_users)

                if not both_users :
                    cosine_dict[item1][item2] = 0
                    adj_cosine_dict[item1][item2] = 0
                    corr_dict[item1][item2] = 0
                else:
                    both_users = list(both_users)
                    numer_cosine = 0
                    denom1_cosine = 0
                    denom2_cosine = 0

                    numer_corr = 0
                    denom1_corr = 0
                    denom2_corr = 0

                    numer_adjusted_cosine = 0
                    denom1_adjusted_cosine = 0
                    denom2_adjusted_cosine = 0
            
                    item1_mean = item_mean_dict[item1]
                    item2_mean = item_mean_dict[item2]

                    for user in both_users:
                        user_mean = user_mean_dict[user]
                        item1_rating = item_dict[item1][user]
                        item2_rating = item_dict[item2][user]

                        a,b,c = cal_cosine(item1_rating, item2_rating)
                        numer_cosine += a
                        denom1_cosine += b
                        denom2_cosine += c

                        a,b,c = cal_correlate(item1_rating, item2_rating, item1_mean, item2_mean)
                        numer_corr +=a
                        denom1_corr += b
                        denom2_corr += c

                        a,b,c = cal_adjusted_cosine(item1_rating, item2_rating, user_mean)
                        numer_adjusted_cosine += a
                        denom1_adjusted_cosine +=b  
                        denom2_adjusted_cosine += c

                    cosine_dict[item1][item2] =  cal_numer_denom(numer_cosine, denom1_cosine, denom2_cosine)
                    corr_dict[item1][item2] =  cal_numer_denom(numer_corr, denom1_corr, denom2_corr)
                    adj_cosine_dict[item1][item2] =  cal_numer_denom(numer_adjusted_cosine,
                                                                     denom1_adjusted_cosine,
                                                                     denom2_adjusted_cosine)
    
    cosine_dict = sort_dict(cosine_dict)
    adj_cosine_dict = sort_dict(adj_cosine_dict)
    corr_dict = sort_dict(corr_dict)
        
    print("time :", time.time() - start)
    return cosine_dict, adj_cosine_dict, corr_dict

In [12]:
def save_file(dict_, path, seed_num, train_rate):
    """
    파일을 저장함
    
    Parameters
    ----------
    dict_ : dict
        item 2개를 2중 key값으로 하여 두 item의 유사도 값을 value 값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    path : dict
        파일 저장 위치 및 이름
    seed_num : int
        랜덤 seed 할당 할 때 썼던 번호
    train_rate : float
        train 데이터 비율로 썼던 값
        
    Returns
    -------
    None
    """
    file_path = '%s_%s_%s.pkl'%(path,train_rate,seed_num)
    with open(file_path, "wb") as json_file:
        pickle.dump(dict_, json_file)

In [13]:
def weighted_sum(test_data_list, tra_user_dict, sim_dict, tra_user_mean_dict, numer_mean_dict='', numer_mean_option ='user', k=0, seed_num=65535):
    """
    유사도와 trian 점수 값을 이용해 예측할 user와 item의 점수 값을 weighted sum으로 산출함
    
    Parameters
    ----------
    test_data_list : list
        [user, item, rating]의 리스트 형태의 객체를 담은 list (test 데이터)
        ex) [ [100, 1000, 3], [101, 1001, 4], ...]
    tra_user_dict : dict
        user 번호를 key값으로, user 번호의 key값에 해당하는 key값을 
        item 놓고 이에 해당하는 점수가 value 값이 됨
        ex) tra_user_dict[1000][100] = 3
    sim_dict : float
        item 2개를 2중 key값으로 하여 두 item의 유사도 값을 value 
        값으로 가지는 dict
        ex) target_dict[1000][1001] = 0.8
    tra_user_mean_dict : dict
        user를 key값으로 하며 대응하는 value 값은 해당 user의 평균 
        rating(점수) 값임 (train 데이터)
    numer_mean_dict : dict
        user(또는 item)를 key값으로 하며 대응하는 value 값은 해당 
        user(item)의 평균 rating(점수) 값임 (train 데이터). The default is ''.
          - numer_mean_option에 따라서 user면 user에 해당하는 mean_dict로,
            item이면 item에 해당하는 mean_dict를 넣어야 함
    numer_mean_option : str
        예측값에 평균값 쓰는 option. The default is ''.
        - '' : 평균값 쓰지 않음
        - item : item 평균값 씀(numer_mean_dict에서 item에 해당하는 
          mean_dict 넣어야 함)
        - user : user 평균값 씀(numer_mean_dict에서 user에 해당하는 
          mean_dict 넣어야 함)
    k : int
        이웃 개수를 정함. 0이면 모든 이웃 사용. The default is 0
    seed_num : int
        랜덤 seed로 사용하는 번호. The default is 65535
    
    Returns
    -------
    pred_list : list
        test_data_list의 점수값 index에 대응하는 예측 점수값
    """
    pred_list = []
    all_mean = 0
    if numer_mean_option !='':
        for key in numer_mean_dict.keys():
            all_mean += numer_mean_dict[key]
        all_mean = all_mean/len(numer_mean_dict.keys())

    
    for user,item,rating in test_data_list:
        if item in sim_dict.keys():
            cand_item_set = sim_dict[item].keys()
        else:
            cand_item_set = []
        user_ok_item = tra_user_dict[user]
        
        both_ok_items = list(set(cand_item_set).intersection(user_ok_item))
        if k != 0:
            temp_list = []
            for t_item in both_ok_items:
                temp_list.append((t_item, sim_dict[item][t_item]))
            temp_list = sorted(temp_list, key=(lambda x: x[1]), reverse=True)
            temp_list = temp_list[:k]
        
        numer = 0
        denom = 0
        plus_mean = 0
        if numer_mean_option == 'user':
            if user not in numer_mean_dict.keys():
                plus_mean = all_mean
            else :
                plus_mean = numer_mean_dict[user]
        elif numer_mean_option == 'item':
            if item not in numer_mean_dict.keys():
                plus_mean = all_mean
            else:
                plus_mean = numer_mean_dict[item]
        

        for t_item in both_ok_items:
            rating = tra_user_dict[user][t_item]
            sim = sim_dict[item][t_item]
            numer += sim*(rating-plus_mean)
            denom += abs(sim)
        
        if denom == 0 :
            pred = plus_mean
        else:
            pred = plus_mean + numer/denom
        
        if pred > 5:
            pred = 5
        elif pred < 1:
            pred =1
        pred_list.append(pred)
    return pred_list   

In [14]:
def extract_error(pred_list, test_data_list):
    """
    true와 pred의 차이값을 추출함. pred와 
    test의 점수값의 index가 같아야 함
    
    Parameters
    ----------
    pred_list : like list
        예측 점수 값을 가진 list
    test_data_list : like list
        [user, item, rating]의 리스트 형태의 객체를 담은 list (test 데이터)
        ex) [ [100, 1000, 3], [101, 1001, 4], ...]
        
    Returns
    -------
    error_list : list
        pred와 true의 차이(절대값)인 error 값을 가진 list
    """
    error_list = []
    for i in range(len(pred_list)):
        pred = pred_list[i]
        
        true = test_data_list[i][2]
        error = abs(pred-true)
        error_list.append(error)
    return error_list

## cross-validation

In [15]:
train_rate = 0.8
seed_num = 65535
inds_train, inds_test = train_test_split(train_rate, df.index.tolist(), seed_num)

train = df.loc[inds_train,:]
test = df.loc[inds_test,:]

### item간 유사도 구하기

In [16]:
error_list = {}
cross_num = 10

for mean_case in ['','user','item']:
    if mean_case == 'user':
        ok_mean_dict = tra_user_mean_dict
    elif mean_case == 'item':
        ok_mean_dict = tra_item_mean_dict
    else:
        ok_mean_dict = ''
        
        
    num_unit = int(len(inds_train)/cross_num)
    t_all_inds_train = inds_train.copy()
    error_list[mean_case] = []
    for i in range(cross_num):
        random.seed(seed_num)
        if i != cross_num-1 :
            t_inds_valid = random.sample(t_all_inds_train, num_unit)

            t_all_inds_train = list(set(t_all_inds_train).difference(t_inds_valid))
        else:
            t_inds_train = t_all_inds_train

        t_inds_train = list(set(inds_train).difference(t_inds_valid))

        data_list = df[['user','item','rating']].values.tolist()
        train = df.loc[t_inds_train,:]
        test = df.loc[t_inds_valid,:]
        
        train_data_list = train[['user','item','rating']].values.tolist()
        test_data_list = test[['user','item','rating']].values.tolist()

        tra_user_dict, tra_item_dict = make_default_dict(train_data_list)
        tes_user_dict, tes_item_dict = make_default_dict(test_data_list)
        tra_user_mean_dict = make_mean_dict(tra_user_dict)
        tra_item_mean_dict = make_mean_dict(tra_item_dict)


        

        cosine_dict, adj_cosine_dict, corr_dict = make_item_sim_dict(tra_item_dict, tra_user_mean_dict, tra_item_mean_dict)
        
        cos_pred_list = weighted_sum(test_data_list, tra_user_dict, cosine_dict, tra_user_mean_dict, ok_mean_dict, mean_case)
        adj_pred_list = weighted_sum(test_data_list, tra_user_dict, adj_cosine_dict, tra_user_mean_dict, ok_mean_dict, mean_case)
        corr_pred_list = weighted_sum(test_data_list, tra_user_dict, corr_dict, tra_user_mean_dict, ok_mean_dict, mean_case)

        cos_error_list = extract_error(cos_pred_list, test_data_list)
        adj_error_list = extract_error(adj_pred_list, test_data_list)
        corr_error_list = extract_error(corr_pred_list, test_data_list)

        cos_mean_error, adj_mean_error, corr_mean_error = np.mean(cos_error_list), np.mean(adj_error_list), np.mean(corr_error_list)
        error_list[mean_case].append((cos_mean_error, adj_mean_error, corr_mean_error))

        save_file(cosine_dict, 'model/cosine_dict_%s_%s'%(mean_case,i), seed_num, train_rate)
        save_file(adj_cosine_dict, 'model/adj_cosine_dict_%s_%s'%(mean_case,i), seed_num, train_rate)
        save_file(corr_dict, 'model/corr_dict_%s_%s'%(mean_case,i), seed_num, train_rate)

        print('%s / %s 완료'%(i+1, cross_num))

time : 58.637728214263916
1 / 10 완료
time : 53.83856129646301
2 / 10 완료
time : 46.95881366729736
3 / 10 완료
time : 43.33609366416931
4 / 10 완료
time : 54.937581300735474
5 / 10 완료
time : 56.7750129699707
6 / 10 완료
time : 52.570063829422
7 / 10 완료
time : 44.64938569068909
8 / 10 완료
time : 51.555914640426636
9 / 10 완료
time : 52.5937774181366
10 / 10 완료
time : 42.41831588745117
1 / 10 완료
time : 42.362539768218994
2 / 10 완료
time : 43.561161518096924
3 / 10 완료
time : 53.375712871551514
4 / 10 완료
time : 51.16243362426758
5 / 10 완료
time : 47.97639322280884
6 / 10 완료
time : 77.57231736183167
7 / 10 완료
time : 57.58895015716553
8 / 10 완료
time : 44.602818965911865
9 / 10 완료
time : 57.045244455337524
10 / 10 완료
time : 45.36640644073486
1 / 10 완료
time : 45.78704476356506
2 / 10 완료
time : 50.21652889251709
3 / 10 완료
time : 46.411176443099976
4 / 10 완료
time : 60.77981972694397
5 / 10 완료
time : 52.59410309791565
6 / 10 완료
time : 45.345951318740845
7 / 10 완료
time : 48.629517555236816
8 / 10 완료
time : 60.2

In [17]:
train = df.loc[inds_train,:]
test = df.loc[inds_test,:]

### Weighted Sum
* cross validation 각 모델에 test data를 넣어 최종 성능 추출

In [20]:
train_rate = 0.8
seed_num = 65535

data_list = df[['user','item','rating']].values.tolist()
train_data_list = train[['user','item','rating']].values.tolist()
test_data_list = test[['user','item','rating']].values.tolist()

tra_user_dict, tra_item_dict = make_default_dict(train_data_list)
tes_user_dict, tes_item_dict = make_default_dict(test_data_list)
tra_user_mean_dict = make_mean_dict(tra_user_dict)
tra_item_mean_dict = make_mean_dict(tra_item_dict)

result_mean_mae_dict = {}
result_mean_pred_mae_dict = {}
pred_dict = {}
for type_ in ['cosine','corr','adj_cosine']:
    result_mean_mae_dict[type_] = {}
    result_mean_pred_mae_dict[type_] = {}
    pred_dict[type_] = {}
    for mean_case in ['','user','item']:
        if mean_case == 'user':
            ok_mean_dict = tra_user_mean_dict
        elif mean_case == 'item':
            ok_mean_dict = tra_item_mean_dict
        else:
            ok_mean_dict = ''

        result_mean_mae_dict[type_][mean_case] = 0
        result_mean_pred_mae_dict[type_][mean_case] = 0
        pred_list = []
        all_pred_list = []
        all_mae = 0
        for i in range(cross_num):
            with open('model/%s_dict_%s_%s_0.8_65535.pkl'%(type_, mean_case,i), 'rb') as f:
                ss = pickle.load(f)
            pred_list = weighted_sum(test_data_list, tra_user_dict, ss, tra_user_mean_dict, ok_mean_dict, mean_case)
            if len(all_pred_list) != 0 :
                all_pred_list = all_pred_list + np.array(pred_list)
            else:
                all_pred_list = np.array(pred_list)
            
            error_list = extract_error(pred_list, test_data_list)
            mae = np.mean(error_list)
            all_mae += mae

        all_mae = all_mae/10
        all_pred_list = all_pred_list/10

        final_error = extract_error(all_pred_list, test_data_list)
        final_mae = np.mean(final_error)
        
        result_mean_mae_dict[type_][mean_case] = all_mae
        result_mean_pred_mae_dict[type_][mean_case] = final_mae
        pred_dict[type_][mean_case] = all_pred_list
    print('%s 완료'%(type_))

cosine 완료
corr 완료
adj_cosine 완료


### 결과 비교
* 전체 결과에서는 Adjusted Cosine, user 평균값을 썼을 때 제일 MAE가 낮았음
* user를 평균값으로 사용할 경우 Adjusted Cosine이 MAE가 제일 낮게 나옴
* item을 평균값으로 사용할 경우 Correlation이 MAE가 제일 낮게 나옴
* 평균값을 사용하지 않을 경우 Cosine이 MAE가 제일 낮게 나옴
* 단, MAE가 낮다고 해서 top-K개 추천시 제일 좋은 성능을 보이는 것은 아닐 수 있음

In [21]:
true_list = [i[2] for i in test_data_list]

In [22]:
print('cosine : %.4f'%(min(result_mean_pred_mae_dict['cosine'].values())))
print('adj_cosine : %.4f'%(min(result_mean_pred_mae_dict['adj_cosine'].values())))
print('corr : %.4f'%(min(result_mean_pred_mae_dict['corr'].values())))

cosine : 0.8330
adj_cosine : 0.7536
corr : 0.7786


In [23]:
result_mean_pred_mae_dict['cosine']

{'': 0.8341879666665798,
 'user': 0.8330383243674998,
 'item': 0.8335478183853131}

In [24]:
result_mean_pred_mae_dict['adj_cosine']

{'': 2.3077362922676903,
 'user': 0.7536008986210845,
 'item': 0.8125025255764494}

In [25]:
result_mean_pred_mae_dict['corr']

{'': 1.5977375100323372,
 'user': 0.8074714793987378,
 'item': 0.7785818293829173}